In [64]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from geopy.geocoders import Nominatim


In [65]:
# Import our input dataset
ny_df = pd.read_csv('column_listings_df.csv', nrows=2000)
ny_df.head()

,listing_id,listing_url,name,host_id,host_url,host_name,host_since,host_is_superhost,host_listings_count,host_total_listings_count,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
0,52702018,https://www.airbnb.com/rooms/52702018,Rental unit in Queens · ★4.90 · 1 bedroom · 2 ...,93231383,https://www.airbnb.com/users/show/93231383,Daniel,2016-09-03,t,3,3,...,2023-08-21,4.90,4.95,4.90,4.95,4.94,4.85,4.87,3,8.53
1,784969376930125242,https://www.airbnb.com/rooms/784969376930125242,Home in Queens · ★4.97 · 3 bedrooms · 4 beds ·...,93231383,https://www.airbnb.com/users/show/93231383,Daniel,2016-09-03,t,3,3,...,2023-08-15,4.97,4.97,4.97,4.95,4.95,4.84,4.87,3,4.47
2,9919728,https://www.airbnb.com/rooms/9919728,Townhouse in Queens · ★4.33 · 3 bedrooms · 7 b...,50997424,https://www.airbnb.com/users/show/50997424,Mark,2015-12-10,f,10,10,...,2020-02-18,4.33,4.17,4.33,4.83,4.67,4.83,4.33,7,0.09
3,53696990,https://www.airbnb.com/rooms/53696990,Vacation home in Queens · Studio · 2 beds · 1 ...,50997424,https://www.airbnb.com/users/show/50997424,Mark,2015-12-10,f,10,10,...,2022-09-16,4.00,4.00,3.00,5.00,4.00,5.00,3.00,7,0.08
4,9920363,https://www.airbnb.com/rooms/9920363,Townhouse in Queens · ★4.25 · 5 bedrooms · 11 ...,50997424,https://www.airbnb.com/users/show/50997424,Mark,2015-12-10,f,10,10,...,2020-01-01,4.25,4.25,3.75,4.75,4.00,4.75,4.50,7,0.09


In [66]:
ny_df.columns

Index(['listing_id', 'listing_url', 'name', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude', 'room_type',
       'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities',
       'price', 'minimum_nights', 'maximum_nights', 'has_availability',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count', 'reviews_per_month'],
      dtype='object')

In [67]:
ny_df=ny_df[['room_type','accommodates', 'bedrooms','neighbourhood_cleansed','price']]

ny_df['bedrooms']= ny_df['bedrooms'].fillna(1)
ny_df['price']= ny_df['price'].str.extract('(\d+\.\d+|\d+)').astype(float)

ny_df.head(3)

,room_type,accommodates,bedrooms,neighbourhood_cleansed,price
0,Entire home/apt,4,1.0,East Elmhurst,171.0
1,Entire home/apt,7,3.0,East Elmhurst,311.0
2,Entire home/apt,12,3.0,Middle Village,328.0


In [68]:
# Encode data
encode_df = pd.get_dummies(ny_df, columns=['room_type', 'neighbourhood_cleansed'], prefix=['room_type', 'neighbourhood_cleansed'])
encode_df.head(3)

,accommodates,bedrooms,price,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,neighbourhood_cleansed_Arverne,neighbourhood_cleansed_Astoria,neighbourhood_cleansed_Bath Beach,neighbourhood_cleansed_Battery Park City,...,neighbourhood_cleansed_Tribeca,neighbourhood_cleansed_Two Bridges,neighbourhood_cleansed_Upper East Side,neighbourhood_cleansed_Upper West Side,neighbourhood_cleansed_Washington Heights,neighbourhood_cleansed_Whitestone,neighbourhood_cleansed_Williamsburg,neighbourhood_cleansed_Windsor Terrace,neighbourhood_cleansed_Woodhaven,neighbourhood_cleansed_Woodside
0,4,1.0,171.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,3.0,311.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,3.0,328.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
encode_df.dtypes

accommodates                                int64
bedrooms                                  float64
price                                     float64
room_type_Entire home/apt                   uint8
room_type_Private room                      uint8
                                           ...   
neighbourhood_cleansed_Whitestone           uint8
neighbourhood_cleansed_Williamsburg         uint8
neighbourhood_cleansed_Windsor Terrace      uint8
neighbourhood_cleansed_Woodhaven            uint8
neighbourhood_cleansed_Woodside             uint8
Length: 105, dtype: object

In [70]:
encode_df = encode_df.dropna()

In [71]:
y = encode_df.price.values
X = encode_df.drop(columns="price").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

LogisticRegression

In [72]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [73]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

LinearRegression()

In [74]:
y_pred = regr.predict(X_test)

In [75]:
# Assuming you already have predictions for the test dataset
y_pred = nn.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

print("Root Mean Squared Error (RMSE):", rmse)

16/16 [==============================] - 0s 1ms/step
Root Mean Squared Error (RMSE): 93.76657914702199


In [76]:
# Score the model
print(f"Training Data Score: {regr.score(X_train, y_train)}")
print(f"Testing Data Score: {regr.score(X_test, y_test)}")

Training Data Score: 0.6688299633974548
Testing Data Score: -9917188980211504.0


#Deep Learning

In [77]:
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=104, activation="relu", input_dim=104))
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1))

nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 104)               10920     
                                                                 
 dense_19 (Dense)            (None, 80)                8400      
                                                                 
 dense_20 (Dense)            (None, 80)                6480      
                                                                 
 dense_21 (Dense)            (None, 80)                6480      
                                                                 
 dense_22 (Dense)            (None, 1)                 81        
                                                                 
Total params: 32361 (126.41 KB)
Trainable params: 32361 (126.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [78]:
nn.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

In [79]:
#fit_model = nn.fit(X_train,y_train,epochs=30)

fit_model = nn.fit(X_train, y_train, epochs=100, validation_split=0.2) 

Epoch 1/100
38/38 [==============================] - 1s 6ms/step - loss: 31918.2695 - mean_absolute_error: 122.6620 - val_loss: 28166.3164 - val_mean_absolute_error: 103.1159
Epoch 2/100
38/38 [==============================] - 0s 3ms/step - loss: 14141.4902 - mean_absolute_error: 70.4089 - val_loss: 13241.7275 - val_mean_absolute_error: 65.4803
Epoch 3/100
38/38 [==============================] - 0s 3ms/step - loss: 10878.0508 - mean_absolute_error: 61.1167 - val_loss: 12876.5869 - val_mean_absolute_error: 68.3537
Epoch 4/100
38/38 [==============================] - 0s 3ms/step - loss: 10581.9121 - mean_absolute_error: 58.4757 - val_loss: 12416.6152 - val_mean_absolute_error: 56.8713
Epoch 5/100
38/38 [==============================] - 0s 3ms/step - loss: 9942.3096 - mean_absolute_error: 54.0994 - val_loss: 12009.6113 - val_mean_absolute_error: 56.5766
Epoch 6/100
38/38 [==============================] - 0s 3ms/step - loss: 9491.2471 - mean_absolute_error: 53.0518 - val_loss: 11645.85

In [80]:
test_loss, test_mae = nn.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

 1/16 [>.............................] - ETA: 0s - loss: 2414.0547 - mean_absolute_error: 36.6652

16/16 [==============================] - 0s 1ms/step - loss: 9467.3193 - mean_absolute_error: 49.6428
Test Loss: 9467.3193359375, Test MAE: 49.64284133911133


In [81]:
# Assuming you already have predictions for the test dataset
y_pred = nn.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

print("Root Mean Squared Error (RMSE):", rmse)

16/16 [==============================] - 0s 920us/step
Root Mean Squared Error (RMSE): 97.3001495105957
